In [1]:
import pandas as pd
import random
from pulp import *

In [2]:
num_lineups = 100
num_overlap = 3
path_skaters = "1_output_players_projections.csv"
path_to_output = "2_output.csv"

def one_lineup_Type_4(skaters, lineups, num_overlap, num_skaters, Center, PG, SG, PF, SF):
    m = LpProblem("NBA_Lineup_Optimization", LpMaximize)
    skaters_lineup = LpVariable.dicts("SkatersLineup", range(num_skaters), 0, 1, LpBinary)

    # Position constraints
    m += lpSum(skaters_lineup[i] for i in range(num_skaters)) == 7, "TotalPlayers"
    m += lpSum(Center[i] * skaters_lineup[i] for i in range(num_skaters)) >= 1, "MinCenter"
    m += lpSum(PG[i] * skaters_lineup[i] for i in range(num_skaters)) >= 1, "MinPG"
    m += lpSum(SG[i] * skaters_lineup[i] for i in range(num_skaters)) >= 1, "MinSG"
    m += lpSum(PF[i] * skaters_lineup[i] for i in range(num_skaters)) >= 1, "MinPF"
    m += lpSum(SF[i] * skaters_lineup[i] for i in range(num_skaters)) >= 1, "MinSF"
    m += lpSum(Center[i] * skaters_lineup[i] for i in range(num_skaters)) <= 3, "MaxCenter"
    m += lpSum(PG[i] * skaters_lineup[i] for i in range(num_skaters)) <= 3, "MaxPG"
    m += lpSum(SG[i] * skaters_lineup[i] for i in range(num_skaters)) <= 3, "MaxSG"
    m += lpSum(PF[i] * skaters_lineup[i] for i in range(num_skaters)) <= 3, "MaxPF"
    m += lpSum(SF[i] * skaters_lineup[i] for i in range(num_skaters)) <= 3, "MaxSF"

    # Salary cap and lineup uniqueness constraints
    m += lpSum(skaters.iloc[i]['Salary'] * skaters_lineup[i] for i in range(num_skaters)) <= 70, "SalaryCap"
    
    # Enforce the maximum overlap constraint for each previous lineup
    for idx, lineup in enumerate(lineups):
        m += lpSum(lineup[i] * skaters_lineup[i] for i in range(num_skaters)) <= num_overlap, f"OverlapConstraint_{idx}"

    # Objective: Maximize projections, modified by a small random factor to introduce variability
    m += lpSum(skaters.iloc[i]['Projection'] * skaters_lineup[i] * (1 + 0.01 * random.random()) for i in range(num_skaters)), "MaximizeProjection"

    # Solve the problem
    m.solve(PULP_CBC_CMD(msg=0))  # Using CBC solver with no output message

    # Extract the lineup
    lineup_result = [1 if skaters_lineup[i].varValue > 0.9 else 0 for i in range(num_skaters)]
    return lineup_result

def create_lineups(num_lineups, num_overlap, path_skaters, formulation, path_to_output):
    skaters = pd.read_csv(path_skaters)
    num_skaters = len(skaters)
    existing_lineups = []
 
    # Generate position flags
    Center = (skaters['Position'] == 'C').astype(int).tolist()
    PG = (skaters['Position'] == 'PG').astype(int).tolist()
    SG = (skaters['Position'] == 'SG').astype(int).tolist()
    PF = (skaters['Position'] == 'PF').astype(int).tolist()
    SF = (skaters['Position'] == 'SF').astype(int).tolist()
 
    # Prepare DataFrame to store all lineups
    player_columns = ['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7']
    projection_columns = ['Points1', 'Points2', 'Points3', 'Points4', 'Points5', 'Points6', 'Points7']
    all_columns = player_columns + projection_columns
    df_lineups = pd.DataFrame(columns=all_columns, index=pd.Index([f"L{i+1}" for i in range(num_lineups)], name="Lineup"))
 
    for i in range(num_lineups):
        lineup = formulation(skaters, existing_lineups, num_overlap, num_skaters, Center, PG, SG, PF, SF)
        existing_lineups.append(lineup)
 
        # Select players and their projections who are in the lineup
        players_in_lineup = [skaters.iloc[idx]['First Name'] for idx in range(num_skaters) if lineup[idx]]
        projections_in_lineup = [skaters.iloc[idx]['Projection'] for idx in range(num_skaters) if lineup[idx]]
 
        # Ensure each lineup has exactly 7 players (if less, fill with "")
        while len(players_in_lineup) < 7:
            players_in_lineup.append("")
        while len(projections_in_lineup) < 7:
            projections_in_lineup.append("")
 
        # Combine player names and projections into a single row in the correct order
        lineup_data = players_in_lineup + projections_in_lineup
 
        # Add the lineup to the DataFrame
        df_lineups.iloc[i] = lineup_data

        #Print while solving
        print(f'Solving Lineup {i+1}')
 
    # Write to CSV file
    df_lineups.to_csv(path_to_output)
 
# Example call
create_lineups(num_lineups, num_overlap, path_skaters, one_lineup_Type_4, path_to_output)

Solving Lineup 1
Solving Lineup 2
Solving Lineup 3
Solving Lineup 4
Solving Lineup 5
Solving Lineup 6
Solving Lineup 7
Solving Lineup 8
Solving Lineup 9
Solving Lineup 10
Solving Lineup 11
Solving Lineup 12
Solving Lineup 13
Solving Lineup 14
Solving Lineup 15
Solving Lineup 16
Solving Lineup 17
Solving Lineup 18
Solving Lineup 19
Solving Lineup 20
Solving Lineup 21
Solving Lineup 22
Solving Lineup 23
Solving Lineup 24
Solving Lineup 25
Solving Lineup 26
Solving Lineup 27
Solving Lineup 28
Solving Lineup 29
Solving Lineup 30
Solving Lineup 31
Solving Lineup 32
Solving Lineup 33
Solving Lineup 34
Solving Lineup 35
Solving Lineup 36
Solving Lineup 37
Solving Lineup 38
Solving Lineup 39
Solving Lineup 40
Solving Lineup 41
Solving Lineup 42
Solving Lineup 43
Solving Lineup 44
Solving Lineup 45
Solving Lineup 46
Solving Lineup 47
Solving Lineup 48
Solving Lineup 49
Solving Lineup 50
Solving Lineup 51
Solving Lineup 52
Solving Lineup 53
Solving Lineup 54
Solving Lineup 55
Solving Lineup 56
S